In [ ]:
!pip install geemap
!pip install earthengine-api
!pip install ee

In [ ]:
!pip install ee

In [ ]:
from io import StringIO

In [ ]:
!pip install ee

In [ ]:
from io import StringIO
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Function to mask edge artifacts in the SAR image
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

# Load Sentinel-1 GRD Image Collection with VV polarization and mask edges
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

# Get a sample of images to display
sample_images = img_vv.limit(5)  # Limit to 5 images for visualization

# Function to add images to the map
def add_images_to_map(collection, map_obj, vis_params, label_prefix):
    for image in collection.getInfo()['features']:
        img = ee.Image(image['id'])
        date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        map_obj.addLayer(img, vis_params, f'{label_prefix} {date}')

# Visualization parameters for Sentinel-1 VV images
vis_params = {
    'min': -30,
    'max': 5,
    'palette': ['black', 'blue', 'cyan', 'green', 'yellow', 'red']
}

# Initialize the map
m = geemap.Map()
m.set_center(5.2013, 47.3277, 12)

# Add Sentinel-1 SAR images to the map
add_images_to_map(sample_images, m, vis_params, 'Sentinel-1 VV')

# Display the map
m

# Separate the descending and ascending orbit images
desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Define seasonal time ranges
spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

# Compute mean images for each season and orbit
desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

# Calculate the change by differencing or ratio
change_detection = asc_change.subtract(desc_change)  # Or use .divide()

# Thresholding to detect significant changes (tunable by the user)
threshold = change_detection.gt(ee.Image.constant(1.5))  # Example threshold

# Integrating ancillary data (e.g., land cover, water bodies) to filter out natural changes
# Example: Using Global Surface Water dataset to mask water-related changes
water_mask = ee.Image("JRC/GSW1_3/GlobalSurfaceWater").select('occurrence').gt(90)
filtered_change = threshold.updateMask(water_mask.Not())  # Exclude water-related changes

# Vectorize the detected man-made changes
change_vector = filtered_change.addBands(change_detection).reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=10,
    maxPixels=1e8
)

# Export the result to a GeoJSON or Shapefile
task = ee.batch.Export.table.toDrive(
    collection=change_vector,
    description='man_made_changes',
    fileFormat='GeoJSON'  # or 'SHP' for Shapefile
)
task.start()